## Part 1: Preprocessing

In [7]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [8]:
# Determine the number of unique values in each column
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [9]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development
...,...,...
1465,No,Research & Development
1466,No,Research & Development
1467,No,Research & Development
1468,No,Sales


In [11]:
# Create a list of at least 10 column names to use as X data
selected_columns = ['Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction',
    'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'NumCompaniesWorked',
    'TotalWorkingYears', 'YearsAtCompany']

# Create X_df using your selected columns
X_df = attrition_df[selected_columns]

# Show the data types for X_df
X_df.dtypes

Age                        int64
DistanceFromHome           int64
Education                  int64
EnvironmentSatisfaction    int64
JobInvolvement             int64
JobLevel                   int64
JobSatisfaction            int64
NumCompaniesWorked         int64
TotalWorkingYears          int64
YearsAtCompany             int64
dtype: object

In [12]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)

In [13]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

encoders = {}
for column in X_train.columns:
    if X_train[column].dtype == 'object':
        encoders[column] = {value: i for i, value in enumerate(X_train[column].unique())}
        X_train[column] = X_train[column].map(encoders[column])
        X_test[column] = X_test[column].map(encoders[column])
        
# Convert the data to numeric types
X_train = X_train.apply(pd.to_numeric)
X_test = X_test.apply(pd.to_numeric)

In [14]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_train_scaled = scaler.fit_transform(X_train)


# Scale the training and testing data
X_test_scaled = scaler.transform(X_test)

In [15]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
department_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_encoded_depart = department_encoder.transform(y_train[['Department']])
y_test_encoded_depart = department_encoder.transform(y_test[['Department']])
y_train_encoded_depart


array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [16]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_encoded_attr = attrition_encoder.transform(y_train[['Attrition']])
y_test_encoded_attr = attrition_encoder.transform(y_test[['Attrition']])
y_train_encoded_attr

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

## Part 2: Create, Compile, and Train the Model

In [17]:
# Find the number of columns in the X training data.
num_columns = X_train_scaled.shape[1]


# Create the input layer
input_layer = layers.Input(shape=(num_columns,))

# Create at least two shared layers 
shared_layer_1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer_2 = layers.Dense(128, activation='relu')(shared_layer_1)


In [18]:
# Create a branch for Department
# with a hidden layer and an output layer


# Create the hidden layer
depart_hidden = layers.Dense(32, activation='relu')(shared_layer_2)

# Create the output layer
depart_output = layers.Dense(y_train_encoded_depart.shape[1], activation='softmax', name='department_output')(depart_hidden)

In [19]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(32, activation='relu')(shared_layer_2)

# Create the output layer for another task (if needed)
attrition_output = layers.Dense(y_train_encoded_attr.shape[1], activation='softmax', name='attrition_output')(attrition_hidden)

In [20]:
# Create the model
model = Model(inputs=input_layer, outputs=[depart_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
            loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'categorical_crossentropy'},
            metrics={'department_output': 'accuracy',
                    'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │        704 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │      8,320 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 32)        │      4,128 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 32)        │      4,128 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ dense_2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         66 │ dense_3[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# Train the model
hystory = model.fit(X_train_scaled,
                {'department_output': y_train_encoded_depart, 
                'attrition_output': y_train_encoded_attr},
                epochs=10,validation_split = 0.2)

Epoch 1/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - attrition_output_accuracy: 0.6759 - attrition_output_loss: 0.5973 - department_output_accuracy: 0.6468 - department_output_loss: 0.9292 - loss: 1.5270 - val_attrition_output_accuracy: 0.8281 - val_attrition_output_loss: 0.5265 - val_department_output_accuracy: 0.6335 - val_department_output_loss: 0.8460 - val_loss: 1.3729
Epoch 2/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8508 - attrition_output_loss: 0.4336 - department_output_accuracy: 0.6814 - department_output_loss: 0.7261 - loss: 1.1599 - val_attrition_output_accuracy: 0.8281 - val_attrition_output_loss: 0.4841 - val_department_output_accuracy: 0.6244 - val_department_output_loss: 0.8379 - val_loss: 1.3228
Epoch 3/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8594 - attrition_output_loss: 0.3744 - department_output_accuracy: 0.6669 - department_output_loss: 0.7699 - loss: 1.1447 - val_attrition_output_accuracy: 0.8281 - val

In [22]:
# Evaluate the model with the testing data
results = model.evaluate(X_test_scaled,
                        {'department_output': y_test_encoded_depart, 
                        'attrition_output': y_test_encoded_attr},
                        )       

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8105 - attrition_output_loss: 0.4348 - department_output_accuracy: 0.6428 - department_output_loss: 0.8280 - loss: 1.2668 


In [23]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {results[3]}")
print(f"Attrition Accuracy: {results[4]}")

Department Accuracy: 0.8288043737411499
Attrition Accuracy: 0.64402174949646


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Is accuracy the best metric to use on this data? Why or why not?
Accuracy may not be the best metric to evaluate the performance of your model, particularly if the dataset is imbalanced. In cases where one class significantly outweighs another, a model can achieve high accuracy simply by predicting the majority class. For example, in an attrition prediction scenario, if 80% of the employees do not leave (have low attrition), predicting "no attrition" for all cases could yield an accuracy of 80%, which does not reflect the model's ability to correctly identify those who do leave. Instead, metrics such as precision, recall, F1 score, or the area under the ROC curve (AUC-ROC) may provide a better understanding of the model's performance, especially if the cost of false positives and false negatives is significant.


2. What activation functions did you choose for your output layers, and why?
The choice of activation functions for output layers typically depends on the nature of the prediction task. For a binary classification problem like attrition prediction, a common choice is the sigmoid activation function. The sigmoid function outputs values between 0 and 1, allowing us to interpret the outputs as probabilities of the positive class (e.g., the probability of attrition). For multi-class problems, a softmax activation function is usually preferred, allowing for the generation of a valid probability distribution across multiple classes. The choice should align with the nature of the output variable (binary vs. multi-class) and interpretability requirements.


3. Can you name a few ways that this model might be improved?
There are several strategies to improve a model’s performance:
  • Data Preprocessing: Ensure data is cleaned, normalized, and properly encoded. Address any missing values and outliers, and consider feature engineering to derive new meaningful features.
  • Class Imbalance Handling: If the data suffers from class imbalance, techniques like oversampling the minority class, undersampling the majority class, or using algorithms like SMOTE (Synthetic Minority Over-sampling Technique) can help.
  • Hyperparameter Tuning: Experiment with different hyperparameters via methods such as grid search or random search to find optimal values that yield better performance.
  • Model Architecture: Adjust the depth and width of the neural network architecture, such as increasing the number of layers or neurons per layer, adding dropout layers to prevent overfitting, or experimenting with different architectures entirely.
  • Regularization Techniques: Apply regularization (L1, L2, or dropout) to reduce overfitting and improve generalization.
  • Ensemble Methods: Combine predictions from multiple models using techniques like bagging or boosting (e.g., Random Forests, Gradient Boosting Machines) to enhance accuracy and robustness.
  • Feature Selection: Identify and retain only the most relevant features to prevent noise from less important variables and improve model interpretability.
  • Model Evaluation: Experiment with different evaluation metrics beyond accuracy to ensure a comprehensive understanding of model performance and areas for improvement.